# Assignment 2 - part-of-speech prediction from limited context

In this assignment, you will train classifiers that attempt, within a window of five words, to make a binary prediction about whether the third word belongs to a given part of speech (noun, verb, adjective, adverb), but using very limited information -- that is, the last two letters of the first, second, fourth, and fifth word of the sequence, and no information whatsoever directly from the third word itself.  You will strip out all punctuation (using the NLTK `WordPunctTokenizer`), lowercase, and remove stop words (using the NLTK English stop words list).

In other words, you will predict over samples that have two classes, P and not-P, where P is the selected part of speech to classify.  For example, from the sentence, "The quick brown fox jumped over the lazy dog.", we can select the following 5-word windows without stop words, "brown fox jumped lazy dog" and "quick brown fox jumped lazy".  If we select verbs as the part-of-speech we are classifying over, we get the instances <(wn,ox,zy,og),1>, since "jumped" is a verb, but <(ck,wn,ed,zy),0> because "fox" in that context is not.

This means that you will need to take into account the position of the last-two-letter feature:  "zy" as the fourth word's last two letters is different from "zy" as the fifth word's last two letters.  They are two features, say, `zy_4` and `zy_5`.

This will likely not actually work.  But it might!

You will create training and testing samples according to this procedure, and you will build a data structure that can be fed to a support vector machine (SVM) classifier.  You will train the classifier on the training data and evaluate it on the testing data. 

The work will be done in a .py module file in the same folder as this notebook.  **No modifications to this notebook will be graded.** We will run your module using this notebook or one we modify that you won't see in order to test your code.

The file you must create and add to the github repo is `mycode.py`, which will be imported here.  You can create your own notebooks or scripts to test it.  You can put any number of your own helper functions and also put optional parameters on any of the python functions mentioned here. You should also create a Markdown file, `notes.md`, to keep any **concise** notes and remarks about the assignment.  The code must run on mltgpu.

**This assignment is due Monday, 2022 March 7, at 23:59. There are 33 points and 5 bonus points.**

In [1]:
import mycode as mc

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/turtilla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/turtilla/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Part 0 - preparation (2 points)

Fork this repository and create and add `mycode.py` and `notes.md`. 

## Part 1 - obtaining the text (3 points)

You will randomly select the given number of lines from the gzipped file we give you (so you will have to figure out how to access gzipped text files).  Explain how you implemented the random selection in `notes.md`. When we run it, it should give a new sample every time.

In [2]:
sampled_lines = mc.sample_lines("UN-english.txt.gz", lines=10000)
                            
print(len(sampled_lines))
sampled_lines[4000:4010]

10000


['Restrictions of passive electoral rights are the same for both genders.',
 'The following cannot be elected to the Saeima (Parliament): persons whose incapacity has been recognised in compliance with the procedure prescribed by law; persons, serving their sentences at penitentiaries; persons who have been sentenced for intentional crimes which are crimes in Latvia also at the moment of the present Law becoming effective; whose criminal record has not been deleted or removed, except persons who have been rehabilitated; persons who have committed a criminal offence in a condition of limited capability or have become mentally ill after the commission of the crime, which has deprived them of the ability to be conscious of their action or to manage it, and who have been imposed a medical coercive measure in view of the disease; or the case has been closed without the imposition of such a coercive measure; persons who are or who have been staff officials of the security services, intellige

## Part 2 - creating the samples (7 points)

From the sampled lines, you will then randomly create the five-word samples.

You will tokenize the sentences and apply POS-tagging to them -- you need to do this before you create the samples, since POS-tagging needs context. You will then remove stop words and punctuation and lowercase the remainder.  Next, you will randomly, over the entire set of sentences, choose samples of five words in sequence, up to a certain limit.  You find the last two characters of the first, second, fourth, and fifth words, and create the type of structure specified up in the introduction to this assignment for each sample. The exact representation is up to you.

In [3]:
processed_sentences = mc.process_sentences(sampled_lines)
processed_sentences[4000:4010]

[[('restrictions', 'NNS'),
  ('passive', 'JJ'),
  ('electoral', 'JJ'),
  ('rights', 'NNS'),
  ('genders', 'NNS')],
 [('following', 'JJ'),
  ('cannot', 'NN'),
  ('elected', 'VBN'),
  ('saeima', 'NNP'),
  ('parliament', 'NNP'),
  ('persons', 'NNS'),
  ('whose', 'WP$'),
  ('incapacity', 'NN'),
  ('recognised', 'VBN'),
  ('compliance', 'NN'),
  ('procedure', 'NN'),
  ('prescribed', 'VBN'),
  ('law', 'NN'),
  ('persons', 'NNS'),
  ('serving', 'VBG'),
  ('sentences', 'NNS'),
  ('penitentiaries', 'NNS'),
  ('persons', 'NNS'),
  ('sentenced', 'VBN'),
  ('intentional', 'JJ'),
  ('crimes', 'NNS'),
  ('crimes', 'NNS'),
  ('latvia', 'NNP'),
  ('also', 'RB'),
  ('moment', 'NN'),
  ('present', 'JJ'),
  ('law', 'NNP'),
  ('becoming', 'VBG'),
  ('effective', 'JJ'),
  ('whose', 'WP$'),
  ('criminal', 'NN'),
  ('record', 'NN'),
  ('deleted', 'VBN'),
  ('removed', 'VBN'),
  ('except', 'IN'),
  ('persons', 'NNS'),
  ('rehabilitated', 'VBN'),
  ('persons', 'NNS'),
  ('committed', 'VBN'),
  ('criminal', 'JJ

In [4]:
all_samples = mc.create_samples(processed_sentences, samples=5000)

print(len(all_samples))
print(all_samples[2500:2510])

5000
[(('rs_1', 'ry_2', 'ee_4', 'on_5'), 0), (('ry_1', 'on_2', 'on_4', 'ee_5'), 0), (('on_1', 'ee_2', 'ee_4', 'er_5'), 0), (('ee_1', 'on_2', 'er_4', 'ry_5'), 0), (('on_1', 'ee_2', 'ry_4', 'ry_5'), 0), (('ee_1', 'er_2', 'ry_4', 'on_5'), 0), (('er_1', 'ry_2', 'on_4', 'nt_5'), 0), (('ry_1', 'ry_2', 'nt_4', 'ls_5'), 0), (('ry_1', 'on_2', 'ls_4', 'so_5'), 0), (('on_1', 'nt_2', 'so_4', 'de_5'), 0)]


## Part 3 - convert the samples into a Pandas DataFrame (10 points)

Here, you will take the samples and create a table whose columns are the features and the class and whose rows are the samples.  All the features and the class will be binary.  Note that there may be many columns, in the hundreds or thousands depending on the diversity of the final two consonants of the non-stop-words in the dataset, but the sum of all rows will be five.

In [5]:
fulldf = mc.create_df(all_samples)

In [7]:
fulldf[2500:2510]

,gh_1,el_2,nt_4,rt_5,el_1,on_2,rt_4,ng_5,on_1,nt_2,...,ri_4,ri_2,ri_1,lp_5,lp_4,eo_5,eo_4,eo_2,eo_1,verb
2500,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2501,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2502,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2503,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2504,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2505,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2506,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2507,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2508,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2509,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


## Part 4 - extract training and testing sets (3 points)

Here, you will create the training and testing datasets in order to train the model.  This will be based on a test percentage.  Round up if the percentage does not divide evenly into the sample size.  You will need to separate the class column into the y-values for the classifier.

In [ ]:
train_X, train_y, test_X, test_y = mc.split_samples(fulldf, test_percent=20)
len(train_X), len(train_y), len(test_X), len(test_y)

## Part 5 - train models (3 points)

You will then train and return two support vector machine (SVM) models using the sklearn SVC class.  You should allow a choice between linear and radial basis function kernels.

In [ ]:
model_linear = mc.train(train_X, train_y, kernel='linear')
model_rbf = mc.train(train_X, train_y, kernel="rbf")
model_linear, model_rbf

## Part 6 - evaluate the models (5 points)

You will calculate and print precision, recall, and F-measure for the models on the test data. In `notes.md`, write down your comparison of these simple measures on the two models and any thoughts you might have on what they mean. (It could be very short, and since the samples do not stay stable between runs, you can save the evaluation scores in `notes.md` too.)

In [ ]:
mc.eval_model(model_linear, test_X, test_y)
mc.eval_model(model_rbf, test_X, test_y)

## Part Bonus - try another sort of model from sklearn (5 points)

Write a separate, command-line script (not a notebook) uses `mycode.py` to do all of the above, except that it trains a non-SVM classifier model.  Any non-trivial model available in sklearn will do. Explain how to run your code and the results of your own evaluation in `notes.md`, including any observations or opinions you may have on the classifier method you used in comparison to SVM.

## Submission

Push to your fork of the GitHub repository (which must be made public) and submit the URL of your repository in Canvas.  You can submit this notebook with the output from your run, as long as you do not modify the code or text in it without permission from us.  